In [1]:
import sys; sys.path.insert(0,'..')
from hashstash.profilers.engine_profiler import *
# logger.setLevel(logging.DEBUG)

In [10]:
stash = HashStash(engine='memory')
f=stash.profiler.profile
fstash=f.stash.clear()

In [18]:
f(iterations=1)
f.stash

[0.90s] function <bound method HashStashProfiler.profile of HashStashProfiler(MemoryHashStash(default_stash / main))> has stash MemoryHashStash(default_stash / stashed_result / hashstash.profilers.engine_profiler.HashStashProfiler.profile)


Config,Param,Value
Path,Root Dir,/Users/ryan/.cache/hashstash
,Name,default_stash
,Dbname,stashed_result/hashstash.profilers.engine_profiler.HashStashProfiler.profile
,Filename,memory.hashstash.zlib+b64.db
Engine,Engine,memory
,Serializer,hashstash
,Compress,True
,B64,True
Misc,Append Mode,True
,Is Function Stash,True


In [19]:
def query_func(d):
    return 'kwargs' in d

In [15]:
f.stash.keys_l()

[{'func': <function hashstash.profilers.engine_profiler.HashStashProfiler.profile(self, size: list = 1000, iterations: int = 1000, num_proc: int = 1, verbose: bool = False, progress: bool = True, **kwargs)>,
  'args': (MemoryHashStash(default_stash / main),),
  'kwargs': {'iterations': 1}}]

In [ ]:
s = HashStash()
stuff(s)

In [ ]:
s['x']=1
stuff(s) == stuff(s)

In [ ]:
stuff(f.__wrapped__)

In [ ]:
stuff(f)

In [ ]:
stuff(f.stash.keys_l()[0]['func'])

In [ ]:
stuff(fstash)

In [ ]:
stuff({'args':(HashStash(),), 'kwargs':{'iterations':1}})

In [ ]:
stash.to_dict()

In [ ]:
key = fstash.keys_l()[0]
key

In [ ]:
key in fstash

In [ ]:
key2b in fstash._keys()

In [ ]:
key2 in fstash

In [ ]:
fstash.has(key)

In [ ]:
fstash._has(fstash.encode_key(key))

In [ ]:
# fstash.clear()
# f(HashStashProfiler(stash), iterations=1,_force=True)

In [ ]:
stash.profiler.profile(iterations=1)

In [ ]:
f(HashStashProfiler(stash), iterations=1)

In [ ]:
key1b = list(fstash._keys())[0]
key1b

In [ ]:
key1b in list(fstash._keys())

In [ ]:
fstash.encode_key(key1) == encode(serialize(key1))

In [ ]:
HashStashProfiler.profile.stash.get_all(10)

In [ ]:
key = {'args': (stash,), 'kwargs': {'iterations': 100}}

In [ ]:
key in HashStashProfiler.profile.stash.keys_l()

In [ ]:
HashStashProfiler.profile.stash.get_all(key)

In [ ]:
stash.profiler.profile_one(data=df)

In [ ]:
c=HashStash()
func = c.profiler.profile
stash = func.stash
df = stash.df
df = df.reset_index().replace({'':np.nan})

# df['Num Proc'] = df['Num Proc'].apply(lambda x: f'{int(x):02}x')
df['Size (KB)'] = df['Raw Size (MB)'].apply(lambda x: int(x*1024))
df['Size (B)'] = df['Raw Size (MB)'].apply(lambda x: int(x*1024*1024))

import math

def group_by_magnitude(size_bytes):
    if size_bytes == 0:
        return '0 B'
    
    magnitude = int(math.log10(size_bytes))
    unit_index = magnitude // 3
    units = ['B', 'KB', 'MB', 'GB', 'TB', 'PB']
    
    if unit_index >= len(units):
        return f'{size_bytes} B'  # Fallback for extremely large sizes
    
    unit = units[unit_index]
    # return unit
    scaled_size = size_bytes / (1024 ** unit_index)
    
    if magnitude % 3 == 0:
        return f'1 {unit}'
    elif magnitude % 3 == 1:
        return f'10 {unit}'
    else:
        return f'100 {unit}'

# Apply the function to create the new column
# df['Size Name'] = df['Size (B)'].apply(lambda x: 10**(round(math.log10(x))//1*1))
df['Size Name'] = df['Size (B)'].apply(lambda x: group_by_magnitude(x))
# df['Size Name'] = df['Size (B)'].apply(lambda x: x//1000*1000)
# df['Size Name'] = df['Size (B)'].apply(lambda x: "B" if x<1000 else "KB" if x<1000000 else "MB")
df['Size Name'].value_counts()
df=df[df['Size Name'].isin({'1 KB'})]

In [ ]:
@stashed_result
def get_speed_stats(group_by:list=None, operations:list=None, carry_over_cols:list=('Speed (MB/s)', 'Time (s)'), also_cols=('Raw Size (MB)',)):
    global df
    # df = df.reset_index().replace({'':np.nan})
    if operations:
        df = df[df['Operation'].isin(operations)]
    
    # Group by Engine and Operation, calculate median for numeric columns
    gby = ([] if not group_by else list(group_by)) + ['Operation']
    grouped = df.groupby(gby).median(numeric_only=True)
    
    # Reset index to make Operation a column
    grouped = grouped.reset_index()
    
    # Pivot the table to get speeds and carried over columns as separate columns
    pivot_values = list(carry_over_cols)
    pivoted = grouped.pivot(index=list(group_by)+list(also_cols), columns='Operation', values=pivot_values)
    
    # Flatten column names
    # pivoted.columns = [f'{col[1]} {col[0]}' if col[0] in carry_over_cols else f'{col[1]} (MB/s)' for col in pivoted.columns]
    pivoted.columns = [f'{col[1]} {col[0].split()[-1]}' for col in pivoted.columns]
    
    # Sort by Total speed descending
    odf = pivoted.reset_index()#.sort_values('Total (MB/s)', ascending=False)
    for c in odf:
        if c.endswith('(s)'):
            odf[f'{c.replace("(s)","(it/s)")}'] = 1/odf[c]
    odf['GetSet (s)'] = odf['Get (s)'] + odf['Set (s)']
    odf['GetSet (it/s)'] = 1/odf['GetSet (s)']
    return odf
get_speed_stats.stash.clear()

In [ ]:

# Call the function and display results
get_speed_stats(group_by=['Engine','Size Name'])

In [ ]:
def plot_speed_comparison(df, group_by=['Engine'], x_col='Write', y_col='Read', color=None, log=False, facet=None):
    import plotnine as p9
    p9.options.figure_size = (8, 6)
    # Prepare the data
    if color and not color in group_by: group_by.append(color)
    if facet and not facet in group_by: group_by.append(facet)
    speed_df = get_speed_stats(group_by=group_by)
    speed_df = speed_df.reset_index()
    
    # Extract the specific operations we want to compare
    # cols = [*group_by]
    # if not x_col in cols: cols.append(x_col)
    # if not y_col in cols: cols.append(y_col)
    plot_df = speed_df #[cols]
    
    # Create a label column that combines all group_by columns
    plot_df['Label'] = plot_df[group_by].apply(lambda row: ' '.join(row.values[:1].astype(str)), axis=1)
    
    # Create the plot
    aes_params = {'x': x_col, 'y': y_col, 'label': 'Label'}
    if color:
        aes_params['color'] = color
    
    plot = (
        ggplot(plot_df, aes(**aes_params))
        # + geom_point(size=3)
        + geom_text(aes(label='Label'), size=8)
        + theme_classic()
        + labs(title=f'{y_col} vs {x_col} Speed Comparison')
            #    x=f'{x_operation} Speed (MB/s)',
            #    y=f'{y_operation} Speed (MB/s)')
    )

    if log:
        plot += scale_x_log10()
        plot += scale_y_log10()
        # plot += labs(x=f'{x_operation} Speed (MB/s) - Log Scale',
                    #  y=f'{y_operation} Speed (MB/s) - Log Scale')

    if facet:
        plot += facet_wrap(facet,ncol=2, scales='free')

    return plot

In [ ]:
comparison_plot = plot_speed_comparison(df, group_by=['Engine'], x_col='Read (it/s)', y_col='Write (it/s)', log=True)
comparison_plot

In [ ]:
comparison_plot = plot_speed_comparison(df, group_by=['Engine'], x_col='Get (it/s)', y_col='Set (it/s)', log=True)
comparison_plot

In [ ]:
comparison_plot = plot_speed_comparison(df, group_by=['Engine'], x_col='Get (it/s)', y_col='Set (it/s)', log=True, color='Data Type')
comparison_plot


In [ ]:
comparison_plot = plot_speed_comparison(df, group_by=['Data Type','Serializer'], x_col='Serialize (it/s)', y_col='Deserialize (it/s)',log=True, color='Serializer')
comparison_plot

In [ ]:
import plotnine as p9
comparison_plot = plot_speed_comparison(df, group_by=['Engine','Num Proc'], x_col='Num Proc', y_col='GetSet (it/s)', log=False, color='Engine') + p9.geom_line()
comparison_plot


In [ ]:
# import plotnine as p9
# comparison_plot = plot_speed_comparison(df, group_by=['Engine','Num Proc'], x_col='Num Proc', y_col='Total (MB/s)', log=False, color='Engine') + p9.geom_line()
# comparison_plot


In [ ]:
import plotnine as p9
comparison_plot = plot_speed_comparison(df, group_by=['Engine','Num Proc'], x_col='Num Proc', y_col='Write (it/s)', log=True, color='Engine') + p9.geom_line() + p9.scale_x_continuous()
comparison_plot


In [ ]:
# import plotnine as p9
# comparison_plot = plot_speed_comparison(df[df.Serializer=='hashstash'], group_by=['Engine','Num Proc'], x_col='Num Proc', y_col='Set (MB/s)', log=True, color='Engine',facet='Size Name') + p9.geom_line()
# comparison_plot
